In [11]:
pip install ollama


Note: you may need to restart the kernel to use updated packages.


# Load input file

In [12]:
project_title_map = {
        "archives_space": "Archives Space Project",
        "archives_space_old": "Archives Space Project",
        "neurohub": "NeuroHub Project",
        "open_spending": "Open Spending Project",
        "open_spending_old": "Open Spending Project",
        "planning_poker": "Planning Poker Project",
        "recycling": "Recycling Project",
        "color_ide": "ColorIDE Project"
    }
projects = ["archives_space", "neurohub", "open_spending", "planning_poker", "recycling", "color_ide"]

selected_project = projects[5] # select which project you want to test with

project_path = "./input_files/user_stories_{}.json".format(selected_project)
with open(project_path, 'r') as file:
        project_content = file.read()
        project_title = project_title_map[selected_project]

# Load model and configure model parameters

In [13]:
import ollama
from ollama import Client

#model_version = "llama3.1:8b-instruct-fp16"
model_version = "llama3.1:70b"
num_ctx = 10240 #context window higher because of the refinement process
temperature = 0 # should be kept low for deterministic results, default value 0.8
num_predict = 1 # we only want 1 token, the score for the given architecture pattern

options = {"num_ctx": num_ctx, "temperature": temperature, "num_predict": 1000}
#model_name = "emir_" + model_version
model_name = model_version
#ollama.create(model=model_name, modelfile=modelfile)

one_shot = False

# Get first assessment from model

In [14]:
client = Client(headers={})

def get_architecture_score(project_title, project_content, architecture_pattern, model_name, options):
    generator_system_message = {'role': 'system', 'content': """
    You are a software architect. Your job is to get a list of categorized user stories with a description, analyze them in detail and
    assign a score for the given architecture pattern depending on their relevance to the project and 
    if it would prove useful in the implementation of the described project.
    IMPORTANT: You MUST respond with ONLY a single digit from 1 to 5, representing the score. DO NOT provide any other text, explanation, introduction or formatting.
    IMPORTANT: The user stories are categorized and given in json format, analyze them really carefully before you answer.
    IMPORTANT: Your answer must relate to the suitability of the given software architecture pattern to the described project.

    EXAMPLE 1:

    Project Title: <title>

    User Stories:

    <user stories>

    Architecture Pattern: Layered Architecture

    Score: <int>

    EXAMPLE 2:

    Project Title: <title>

    User Stories:

    <user stories>

    Architecture Pattern: Microservices Architecture

    Score: <int>
    """}
    user_message = {'role': 'user', 'content': f"""
    Project Title: {project_title}

    User Stories:

    {project_content}

    Architecture Pattern: {architecture_pattern}

    Score: 
    """}
    messages = [generator_system_message, user_message]  # Reset messages for each pattern

    response = client.chat(model=model_name, messages=messages, options=options)
    message = response['message']
    print(message['content'])

    try:
        score = int(message['content'].strip())
        if 1 <= score <= 5:
            return score
        else:
            print(f"Invalid score: {score}. Model returned a number outside the range 1-5.")
            return None  # Or raise an exception
    except ValueError:
        print(f"Invalid response: '{message['content']}'. Model did not return a valid integer.")
        return None  # Or raise an exception

In [15]:
architecture_patterns = ["Layered Architecture", "Event-Driven Architecture (Pub-Sub Architecture)",
 "Microkernel Architecture", "Microservices Architecture", "Space-Based Architecture", "Pipeline Architecture (Pipe-Filter Architecture)", "Client-Server Architecture"]

for pattern in architecture_patterns:
    score = get_architecture_score(project_title, project_content, pattern, model_name, options)
    if score is not None:
        print(f"Score for {pattern}: {score}")

4
Score for Layered Architecture: 4
4
Score for Event-Driven Architecture (Pub-Sub Architecture): 4
4
Score for Microkernel Architecture: 4
4
Score for Microservices Architecture: 4
4
Score for Space-Based Architecture: 4
4
Score for Pipeline Architecture (Pipe-Filter Architecture): 4
4
Score for Client-Server Architecture: 4
